In [2]:
from DeepLearning.Environments.CatanEnv import CatanEnv
from DeepLearning.PPO import MaskablePPO
from DeepLearning.GetActionMask import getActionMask
from DeepLearning.GetObservation import getObservation, getObservationSimplified, lowerBoundsSimplified, upperBoundsSimplified
from Agents.AgentModel import AgentMultiModel
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentNoMoves import AgentNoMoves
import os


os.environ["UPDATE_MODELS"] = "False"
os.environ["MODEL_NAME"] = "None"
os.environ["VP_REWARDS"] = "False"
os.environ["WIN_REWARDS"] = "False"

env = CatanEnv(players=[AgentRandom2("P0", 0),
                        AgentRandom2("P1", 1),
                        AgentRandom2("P2", 2),
                        AgentRandom2("P3", 3),])

info = {
    "env": "CatanEnv",
    "Timesteps": "1M",
    "Opponents": "3xAgentRandom2",
    "Rewards": "Setup, Dense, Bank Trades"
}
name = "Gamma_9"

netArchDict = dict(pi=[128, 128], vf=[128, 128])

model = MaskablePPO("MlpPolicy", env, policy_kwargs=dict(net_arch=netArchDict), gamma=0.9, verbose=1, getActionMask=getActionMask, getObservation=getObservationSimplified, info=info, saveName=name, tensorboard_log="./tensorboard_logs/")
# model=MaskablePPO.load("DeepLearning/Models/Full/Rewards_SetupDenseTrades_NoDevCard.zip", env=env)
# model.saveName = name
model.learn(total_timesteps=500_000, tb_log_name=f"{name}")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard_logs/Gamma_4_1


/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 84.9     |
|    ep_rew_mean     | -20      |
| time/              |          |
|    fps             | 1302     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:283: UserWarning: Path 'DeepLearning/models/Gamma_4' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 88.4        |
|    ep_rew_mean          | -4.1        |
| time/                   |             |
|    fps                  | 909         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.019172104 |
|    clip_fraction        | 0.28        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.91       |
|    explained_variance   | 0.00227     |
|    learning_rate        | 0.0003      |
|    loss                 | 22.3        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0413     |
|    value_loss           | 30.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 87.4  

In [1]:
# Test Trained model using env format, use to debug running environments 

from sb3_contrib.common.maskable.utils import get_action_masks
from DeepLearning.Environments.NoSetupEnv import NoSetupDenseRewardEnv, NoSetupEnv
from DeepLearning.Environments.SetupEnv import SetupRandomWithRoadsEnv
from DeepLearning.Environments.CatanEnv import CatanEnv
from DeepLearning.Environments.SelfPlayEnv import SelfPlayEnv, SelfPlaySetupDotTotalEnv
import os
from Game.CatanPlayer import PlayerStatsTracker
from DeepLearning.PPO import MaskablePPO
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentNoMoves import AgentNoMoves
from Agents.AgentMCTS import AgentMCTS
from tabulate import tabulate
from DeepLearning.Stats import headers
import pandas as pd
import random
from CatanData.GameStateViewer import SaveGameStateImage, DisplayImage
import time


os.environ["UPDATE_MODELS"] = "False"
os.environ["MODEL_NAME"] = "None"

model=MaskablePPO.load("DeepLearning/Models/Full/Rewards_SetupDenseTrades_NoDevCard.zip")

rewardList = []
winner = [0,0,0,0]

players = [ AgentRandom2("P0", 0),
            AgentNoMoves("P1", 1),
            AgentNoMoves("P2", 2),
            AgentNoMoves("P3", 3),]

env = CatanEnv(players=players)

total_actions = 0

for episode in range(1):
    done = False
    state, info = env.reset()#players=players)

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        rewardList.append(reward)
        total_actions += 1

    winner[env.game.gameState.winner] += 1
    
    # env.game.gameState.players[0].generatePlayerStats()
    # env.game.gameState.players[3].generatePlayerStats()

    # print(env.game.gameState.players[0].stats)
    # DisplayImage(env.game.gameState)
    # print(rewardList[-1] - 25)
print(rewardList)
print(sum(rewardList))

print(f"TotalActions:{total_actions}")
print("\n\nWinnings: ", winner)


# Brick, ore, wool, wheat, wood

[0, 0, 13.0, 0, 0, 0, 0, -0.25, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.25, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.25, 0, 0, 0, 0, 0, 0, 0, -0.25, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 4, 0, -0.25, 0, 0, 4, 0, 0, 4, 4, 0, -0.25, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0]
61.5
TotalActions:137


Winnings:  [1, 0, 0, 0]


In [3]:
"""
Running Agent simulations
"""
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentMCTS import AgentMCTS
from Agents.AgentUCT import AgentUCT
from Agents.AgentNoMoves import AgentNoMoves
from Agents.AgentModel import AgentMultiModel, AgentModel
from Game.CatanGame import *
from CatanSimulator import CreateGame
from DeepLearning.PPO import MaskablePPO
from Game.CatanPlayer import PlayerStatsTracker
from tabulate import tabulate
from DeepLearning.Stats import headers
import dill as pickle
import pandas as pd
from CatanData.GameStateViewer import SaveGameStateImage, DisplayImage
import time
import math
from DeepLearning.GetObservation import getObservationSimplified

# best_model = AgentMultiModel("P1", 1, model=MaskablePPO.load("DeepLearning/Models/NoSetup/NoSetupDenseRewardEnv-10M.zip"), setupModel=MaskablePPO.load("DeepLearning/Models/Setup/SetupRandom_wins_1M.zip"), fullSetup=False)

winner = [0,0,0,0]
player0Stats = PlayerStatsTracker()
Player0LosingStats = PlayerStatsTracker()
player1Stats = PlayerStatsTracker()

players = [ AgentModel("P0", 0, model=MaskablePPO.load("DeepLearning/Models/Gamma_4/Gamma_4_491520.zip")),
            AgentRandom2("P1", 1),
            AgentRandom2("P2", 2),
            AgentRandom2("P3", 3),]

COLLECT_STATS = True
numTurns = 0
for episode in range(1000):
    game = CreateGame(players)
    game = pickle.loads(pickle.dumps(game, -1))
    while True:
        currPlayer = game.gameState.players[game.gameState.currPlayer]

        agentAction = currPlayer.DoMove(game)
        agentAction.ApplyAction(game.gameState)

        # if currPlayer.seatNumber == 0: # and agentAction.type == 'EndTurn':
        #     DisplayImage(game.gameState, agentAction)
        #     time.sleep(1)
        #     numTurns += 1

        if game.gameState.currState == "OVER":
            break
    
    # print("Winner: ", game.gameState.winner)
    winner[game.gameState.winner] += 1
    lost = game.gameState.winner != 0

    # Stats
    if COLLECT_STATS:
        game.gameState.players[0].generatePlayerStats()
        game.gameState.players[1].generatePlayerStats()

        player0Stats += game.gameState.players[0].stats
        player1Stats += game.gameState.players[1].stats
        if lost:
            Player0LosingStats += game.gameState.players[0].stats

# Collect stats
if COLLECT_STATS:
    player0Stats.getAverages()
    Player0LosingStats.getAverages()
    player1Stats.getAverages()
    player0Data = player0Stats.getList()
    player0LosingData = Player0LosingStats.getList()
    player1Data = player1Stats.getList()

    p_hat0 = winner[0] / sum(winner)
    p_hat1 = winner[1] / sum(winner)
    margin_error0 = round(100*(1.96 * math.sqrt((p_hat0 * (1 - p_hat0)) / sum(winner))), 2)
    margin_error1 = round(100*(1.96 * math.sqrt((p_hat1 * (1 - p_hat1)) / sum(winner))), 2)
    player0Data.insert(0, margin_error0)
    player0LosingData.insert(0, -1)
    player1Data.insert(0, margin_error1)
    player0Data.insert(0, winner[0]/sum(winner))
    player0LosingData.insert(0, -1)
    player1Data.insert(0, winner[1]/sum(winner))
    player0Data.insert(0, "Player0")
    player0LosingData.insert(0, "Player0LossesStats")
    player1Data.insert(0, "Player1")

    table = tabulate([player0Data, player0LosingData, player1Data], headers=headers, tablefmt='simple')
    print(table)

print(f"\nNum turns: {numTurns}")

print("\n\nWinnings: ", winner)


# Brick, ore, wool, wheat, wood

AgentName             WinRate    MarginError    numTurns    victoryPoints    numRoadsBuilt    devCardsBought  usedDevCards                         settlementsBuilt    citiesBuilt    devCardVP    largestArmy    longestRoad  resourcesReceived                          totalResourcesReceivedPerTurn    totalResourcesDiscarded    totalResourcesStolen  resourcesFromDevCard                   totalResourcesFromDevCard  resourcesFromBankTrade                  totalResourcesFromBankTrade  finalResourceProduction                finalTradeRates                                                                                    setupResourceProduction                totalSetupResourceProduction  setupTradeRates                        setupResourceDiversity    turnsForFirstSettlement    noSettlementsBuilt    turnsForFirstCity    noCitysBuilt
------------------  ---------  -------------  ----------  ---------------  ---------------  ----------------  ---------------------------------  -----------------

In [7]:
# # Save to csv
# fileName = f'Rewards_SetupDenseTradesLosses_3M_v_3AgentRandom.csv'
# df = pd.DataFrame([player0Data, player0LosingData, player1Data], columns=headers)
# df.to_csv(f'DeepLearning/Data/Full/{fileName}', index=False)

# from DeepLearning.GetActionMask import allActionsDict

# print(allActionsDict)

# model.save("DeepLearning/Models/Rewards_SetupDenseTradesLosses/Final")

